# FIND DUPLICATES

## Import the files

In [36]:
import pandas as pd
import csv
import os
from datetime import datetime

# Define the data types for the columns
dtype_dict = {
    'ECLID': str,
    'First_Occurrence_ECLID': str,
    'COMPANY': str,
    'UPC': str,
    'PUID1': str
}

# Function to get the last modified timestamp of a file
def get_file_timestamp(file_path):
    timestamp = os.path.getmtime(file_path)  # Get the last modified time in seconds since epoch
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')  # Convert to readable format

# Function to log messages to both the console and a log file
def log_message(message, log_file='C:/Users/mark.artim/OneDrive - Heritage Distribution Holdings/EclipseDownload/importlog.txt'):
    # Get the current timestamp for the log entry
    log_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{log_timestamp}] {message}\n"

    # Print to the console
    print(log_entry, end='')

    # Append to the log file
    with open(log_file, 'a') as file:
        file.write(log_entry)

# File paths
file_path_HER = 'C:/Users/mark.artim/OneDrive - Heritage Distribution Holdings/EclipseDownload/ProdIDQualityHER.csv'
file_path_EDS = 'C:/Users/mark.artim/OneDrive - Heritage Distribution Holdings/EclipseDownload/ProdIDQualityEDS.csv'

# Get timestamps
timestamp_HER = get_file_timestamp(file_path_HER)
timestamp_EDS = get_file_timestamp(file_path_EDS)

# Load CSV files
data1 = pd.read_csv(file_path_HER, encoding='windows-1252', skiprows=8, dtype=dtype_dict)
data1.insert(0, 'COMPANY', 'A-HER')

data2 = pd.read_csv(file_path_EDS, encoding='windows-1252', skiprows=8, dtype=dtype_dict)
data2.insert(0, 'COMPANY', 'B-EDS')

# Count the number of rows in the CSV files
with open(file_path_HER, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count_HER = sum(1 for row in reader) - 8  # Subtract 8 to account for skipped rows

with open(file_path_EDS, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count_EDS = sum(1 for row in reader) - 8  # Subtract 8 to account for skipped rows

# Log timestamps and row counts
log_message(f"HER CSV file last modified: {timestamp_HER}")
log_message(f"Number of rows in HER CSV file: {row_count_HER}")

log_message(f"EDS CSV file last modified: {timestamp_EDS}")
log_message(f"Number of rows in EDS CSV file: {row_count_EDS}")

# Append the DataFrames
data = pd.concat([data1, data2], ignore_index=True)

# Make a copy of the dataframe for finding matches on PUID1 which is Catalog Number!!
datac = data.copy()

# Log the number of records for EDS versus HER
data_EDS = data[(data['COMPANY'] == 'B-EDS')]
data_HER = data[(data['COMPANY'] == 'A-HER')]
log_message(f"Number of records found for HER in combined dataframe 'data': {len(data_HER)}")
log_message(f"Number of records found for EDS in combined dataframe 'data': {len(data_EDS)}")
log_message(f"Total Number of records in dataframe 'data': {len(data)}")

[2025-02-09 19:43:20] HER CSV file last modified: 2025-02-07 17:06:55
[2025-02-09 19:43:20] Number of rows in HER CSV file: 44147
[2025-02-09 19:43:20] EDS CSV file last modified: 2025-02-07 17:07:43
[2025-02-09 19:43:20] Number of rows in EDS CSV file: 76524
[2025-02-09 19:43:21] Number of records found for HER in combined dataframe 'data': 44146
[2025-02-09 19:43:21] Number of records found for EDS in combined dataframe 'data': 76523
[2025-02-09 19:43:21] Total Number of records in dataframe 'data': 120669


## Find UPC Matches

In [38]:
#data.describe()
#data.info()

print(f"Total Number of records in dataframe 'data': {len(data)}")

# Calculate the value counts for the 'UPC' column
UPC_count = data['UPC'].value_counts()

# Map the value counts & MatchType back to the DataFrame
data['UPC_Count'] = data['UPC'].map(UPC_count)
data['MatchType'] = 'UPCMATCH'

# Insert the new column at the specified position
data.insert(6, 'UPC_Count', data.pop('UPC_Count'))

# SHOW HOW MANY XREFS BASED ON UPC
# Condition to check UPC_count > 1
condition = data['UPC_Count'] > 1
# Count the number of rows that meet the condition
count = condition.sum()
# Print the count
print(f"Number of rows where UPC_count > 1: {count}")

# Print the number of records found
print(f"Number or rows in data: {len(data)}")

# Create a new column with concatenated values for sorting HER products before Ed's
data['Company_UPC'] = data['UPC'] + '-' + data['COMPANY'].astype(str)

data['ECLID_INT'] = pd.to_numeric(data['ECLID'], downcast ='signed')
data = data.sort_values(by=['Company_UPC', 'ECLID_INT'], ascending=True)

# Create a dictionary to store the first occurrence of each UPC
first_occurrence = {}
first_occurrence_co = {}
for index, row in data.iterrows():
    if row['UPC'] not in first_occurrence:
        first_occurrence[row['UPC']] = row['ECLID']
        first_occurrence_co[row['UPC']] = row['COMPANY']

# Add a new column to the data based on the first occurrence
data['First_Occurrence_ECLID'] = data['UPC'].map(first_occurrence)
data['First_Occurrence_COMPANY'] = data['UPC'].map(first_occurrence_co)
upcxref = data

# Define the CSV file name
upc_xref_file_name = 'C:/Users/mark.artim/Documents/upc.pn.xref.csv'

# CREATE THE PN XREF FILE
print(f"Now creating the UPC cross reference file {upc_xref_file_name}")

data = data[data['COMPANY'] == 'B-EDS']
print(f"Filter for only Ed's records. Number of records: {len(data)}")
data = data[data['UPC_Count'] > 1]
print(f"Filter for only UPC_Count > 1. Number of records: {len(data)}")

#data1 = pd.read_csv(file_path, encoding='windows-1252', skiprows=8)
selected_columns = data[['ECLID', 'First_Occurrence_ECLID', 'COMPANY', 'First_Occurrence_COMPANY', 'DESC', 'UPC', "UPC_Count", 'BUYLINE', 'MatchType','BUYLINEX']]
#selected_columns['MatchType'] = 'UPCMATCH'
# Save the selected columns to a new CSV file
selected_columns.to_csv(upc_xref_file_name, index=False)

print(f"The new CSV file, {upc_xref_file_name}, has been created.")

print(selected_columns.head(2))




Total Number of records in dataframe 'data': 120669
Number of rows where UPC_count > 1: 27101
Number or rows in data: 120669
Now creating the UPC cross reference file C:/Users/mark.artim/Documents/upc.pn.xref.csv
Filter for only Ed's records. Number of records: 76523
Filter for only UPC_Count > 1. Number of records: 17712
The new CSV file, C:/Users/mark.artim/Documents/upc.pn.xref.csv, has been created.
        ECLID First_Occurrence_ECLID COMPANY First_Occurrence_COMPANY  \
79795  335684                  77295   B-EDS                    A-HER   
79786   21187                  12897   B-EDS                    A-HER   

                                   DESC          UPC  UPC_Count BUYLINE  \
79795  VCMA-20ULS-PRO 230V LITTLE GIANT  01012101443        2.0     LTG   
79786     RIM-U LG DUAL V UNIV I-M PUMP  01012110705        2.0     LTG   

      MatchType BUYLINEX  
79795  UPCMATCH      LTG  
79786  UPCMATCH      LTG  


## Alternate UPC Match Logic

In [32]:
# Calculate the value counts for the 'UPC' column
data['UPC_Count'] = data['UPC'].map(data['UPC'].value_counts())

# Create a dictionary to store the first occurrence of each UPC
first_occurrence_dict = data.drop_duplicates(subset='UPC').set_index('UPC')['ECLID']

# Map the first occurrence to the data
data['First_Occurrence_ECLID'] = data['UPC'].map(first_occurrence_dict)
data['First_Occurrence_COMPANY'] = data['UPC'].map(first_occurrence_co)

# Add a match type column
data['MatchType'] = 'UPCMATCH'

# Filter the data for `B-EDS` and `UPC_Count > 1`
filtered_data = data[(data['COMPANY'] == 'B-EDS') & (data['UPC_Count'] > 1)]

# Select the relevant columns for the output file
selected_columns = filtered_data[
    [
        'ECLID',
        'First_Occurrence_ECLID',
        'COMPANY',
        'First_Occurrence_COMPANY',
        'DESC',
        'UPC',
        "UPC_Count",
        'BUYLINE',
        'MatchType',
    ]
]

# Define the output file path
upc_xref_file_name = 'C:/Users/mark.artim/Documents/upc.pn.xref.csv'

# Save the filtered data to the output file
selected_columns.to_csv(upc_xref_file_name, index=False)

print(f"The new CSV file, {upc_xref_file_name}, has been created.")

# Count the number of rows in the CSV file cat.pn.xref.csv AFTER removing records in UPC file
with open(upc_xref_file_name, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)

print(f"Number of rows in upc.pn.xref.csv file: {row_count}")



The new CSV file, C:/Users/mark.artim/Documents/upc.pn.xref.csv, has been created.
Number of rows in upc.pn.xref.csv file: 17713


## Find CATALOG NUMBER Matches

In [40]:
print(f"Total Number of records in dataframe 'datac': {len(datac)}")

# Calculate the value counts for the 'UPC' column
CAT_count = datac['PUID1'].value_counts()

#Map the value counts & match type back to the DataFrame
datac['CAT_Count'] = datac['PUID1'].map(CAT_count)
datac['MatchType'] = 'CATALOGMATCH'

# Print the number of records found
print(f"Number or rows in data: {len(datac)}")

# Create a new column with concatenated values for sorting HER products before Ed's
datac['Company_CAT'] = datac['PUID1'] + '-' + datac['COMPANY'].astype(str)
datac['ECLID_INT'] = pd.to_numeric(datac['ECLID'], downcast ='signed')
datac['CAT_BUYLINE'] = datac['PUID1'] + '-' + datac['BUYLINE']

# Sort by the new column
datac = datac.sort_values(by=['CAT_BUYLINE', 'ECLID_INT'])
print("datac first record:")


# Calculate the value counts for the 'PUID1_BUYLINE' column
CAT_Count = datac['CAT_BUYLINE'].value_counts()

# Map the value counts back to the DataFrame
#datac['CAT_Count'] = datac['CAT_BUYLINE'].map(CAT_Count)

# Insert the new column at the specified position
datac.insert(3, 'CAT_Count', datac.pop('CAT_Count'))
datac.insert(4, 'CAT_BUYLINE', datac.pop('CAT_BUYLINE'))
print(datac.head(1))


# Create a dictionary to store the first occurrence of each UPC
first_occurrence = {}
first_occurrence_co = {}
for index, row in datac.iterrows():
    if row['CAT_BUYLINE'] not in first_occurrence:
        first_occurrence[row['CAT_BUYLINE']] = row['ECLID']
        first_occurrence_co[row['CAT_BUYLINE']] = row['COMPANY']

# Add a new column to the data based on the first occurrence
datac['First_Occurrence_ECLID'] = datac['CAT_BUYLINE'].map(first_occurrence)
datac['First_Occurrence_COMPANY'] = datac['CAT_BUYLINE'].map(first_occurrence_co)
catxref = datac

# Define the CSV file name
cat_xref_file_name = 'C:/Users/mark.artim/Documents/cat.pn.xref.csv'

# CREATE THE PN XREF FILE
print(f"Now creating the UPC cross reference file {cat_xref_file_name}")

datac = datac[datac['COMPANY'] == 'B-EDS']
print(f"Filter for only Ed's records. Number of records: {len(datac)}")
datac = datac[datac['CAT_Count'] > 1]
print(f"Filter for only CAT_Count > 1. Number of records: {len(datac)}")

#data1 = pd.read_csv(file_path, encoding='windows-1252', skiprows=8)
selected_columns = datac[['ECLID', 'First_Occurrence_ECLID', 'COMPANY', 'First_Occurrence_COMPANY', 'DESC', 'PUID1', "CAT_Count", 'BUYLINE', 'MatchType','BUYLINEX']]
#selected_columns['MatchType'] = 'CATALOGMATCH'
# Save the selected columns to a new CSV file
selected_columns.to_csv(cat_xref_file_name, index=False)

print(f"The new CSV file, {cat_xref_file_name}, has been created.")

print(selected_columns.head(2))

print(f"The new CSV file, {cat_xref_file_name}, has been created.")

# Count the number of rows in the CSV file cat.pn.xref.csv AFTER removing records in UPC file
with open(cat_xref_file_name, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)

print(f"Number of rows in cat.pn.xref.csv file: {row_count}")


Total Number of records in dataframe 'datac': 120669
Number or rows in data: 120669
datac first record:
  COMPANY BUYLINE  ECLID  CAT_Count            CAT_BUYLINE  \
6   A-HER  3TCORP  94157        1.0  3TCORP-1624121-3TCORP   

                                DESC  UPC           PUID1       PUID1A PUID2  \
6  3TC.1624121 16 X 24 1/2 X 1 METAL  NaN  3TCORP-1624121  3TC.1624121   NaN   

   UPCLGTH    STATUS PRCLINE BUYLINEX     MatchType           Company_CAT  \
6      0.0  NONSTOCK  3TCORP      NaN  CATALOGMATCH  3TCORP-1624121-A-HER   

   ECLID_INT  
6    94157.0  
Now creating the UPC cross reference file C:/Users/mark.artim/Documents/cat.pn.xref.csv
Filter for only Ed's records. Number of records: 76523
Filter for only CAT_Count > 1. Number of records: 22183
The new CSV file, C:/Users/mark.artim/Documents/cat.pn.xref.csv, has been created.
        ECLID First_Occurrence_ECLID COMPANY First_Occurrence_COMPANY  \
44153   99187                  99187   B-EDS                    B-EDS 

## Remove entries in the CATALOG match file that are already in the UPC file

In [42]:
import pandas as pd
import csv

# Load the two CSV files
cat_file = "C:/Users/mark.artim/Documents/cat.pn.xref.csv"
upc_file = "C:/Users/mark.artim/Documents/upc.pn.xref.csv"

# Count the number of rows in the CSV file cat.pn.xref.csv
with open(cat_file, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)
print(f"Number of rows in cat.pn.xref.csv file: {row_count}")

cat_df = pd.read_csv(cat_file, header=None)  # Load cat.pn.xref.csv
upc_df = pd.read_csv(upc_file, header=None)  # Load upc.pn.xref.csv

# Assuming the first column in both files is column 0
cat_first_column = cat_df[0]
upc_first_column = upc_df[0]

# Filter rows in cat_df where the first column value is NOT in the upc first column
filtered_cat_df = cat_df[~cat_df[0].isin(upc_first_column)]

# Save the filtered data back to the same file or a new file
new_cat_file = "C:/Users/mark.artim/Documents/filtered_cat.pn.xref.csv"
filtered_cat_df.to_csv(new_cat_file, index=False, header=False)

print("Records removed. Filtered file saved as 'filtered_cat.pn.xref.csv'")

# Count the number of rows in the CSV file cat.pn.xref.csv AFTER removing records in UPC file
with open(new_cat_file, 'r', encoding='windows-1252') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)

print(f"Number of rows in cat.pn.xref.csv file: {row_count}")


Number of rows in cat.pn.xref.csv file: 22184
Records removed. Filtered file saved as 'filtered_cat.pn.xref.csv'
Number of rows in cat.pn.xref.csv file: 9499
